# Manage Redshift Cluster

This notebook contains code snippets for creating and tearing down a Redshift cluster for the project.

In [73]:
import os
import time
from configparser import ConfigParser
import json
import boto3
from botocore.exceptions import ClientError
import pandas as pd

In [66]:
user = ConfigParser()
user.read('user.cfg')
ACCESS_KEY = user.get('AWS_USER', 'AWS_ACCESS_KEY_ID')
SECRET = user.get('AWS_USER', 'AWS_SECRET_ACCESS_KEY')

In [67]:
config = ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

## Create Cluster

In [19]:
redshift = boto3.client(
    'redshift', 
    region_name='us-west-2', 
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET,
)

In [20]:
response = redshift.create_cluster(
    ClusterIdentifier=config.get('CLUSTER', 'CLUSTER_ID'),
    ClusterType=config.get('CLUSTER', 'CLUSTER_TYPE'),
    NodeType=config.get('CLUSTER', 'NODE_TYPE'),
    NumberOfNodes=int(config.get('CLUSTER', 'NUMBER_OF_NODES')),
    DBName=config.get('CLUSTER', 'DB_NAME'),
    Port=int(config.get('CLUSTER', 'DB_PORT')),
    MasterUsername=config.get('CLUSTER', 'DB_USER'),
    MasterUserPassword=config.get('CLUSTER', 'DB_PASSWORD'),
    IamRoles=[config.get('IAM_ROLE', 'ARN')],
)

In [35]:
cluster = response['Cluster']

In [38]:
# Poll for cluster status every ~30 sec
counter = 0
while True:
    cluster = (
        redshift
        .describe_clusters(ClusterIdentifier=config.get('CLUSTER', 'CLUSTER_ID'))
        ['Clusters'][0]
    )
    if cluster['ClusterStatus'] == 'available':
        print('Cluster is available')
        break
    else:
        counter += 1        
        print(f'ClusterStatus after {counter} tries: {cluster["ClusterStatus"]}')
        time.sleep(30)

Cluster is available


In [57]:
cluster

{'ClusterIdentifier': 'dend-1',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'MasterUsername': 'awsuser',
 'DBName': 'dend-dwh-project',
 'Endpoint': {'Address': 'dend-1.cgepvlalvgqs.us-west-2.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2020, 10, 9, 18, 20, 11, 95000, tzinfo=tzlocal()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-513e617c',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-bd0449c5',
 'AvailabilityZone': 'us-west-2c',
 'PreferredMaintenanceWindow': 'tue:07:00-tue:07:30',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 4,
 'PubliclyAccessible': True,
 'Encrypted': False,
 'ClusterPublicKey': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwRhr2cJCOdpBPxKPcZX

## Set TCP port

In [68]:
ec2 = boto3.resource(
    'ec2',
    region_name="us-west-2",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET,
)

In [69]:
sec_group_id = cluster['VpcSecurityGroups'][0]['VpcSecurityGroupId']
sec_group = ec2.SecurityGroup(sec_group_id)

In [75]:
try:
    sec_group.authorize_ingress(
        GroupName=sec_group.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(config.get('CLUSTER', 'DB_PORT')),
        ToPort=int(config.get('CLUSTER', 'DB_PORT'))
    )
except ClientError as e:
    print(e)

An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## Save Connection Details

This JSON will be used in `create_table.py` and `etl.py` scripts.

In [48]:
DWH_ENDPOINT = cluster['Endpoint']['Address']
DWH_PORT = config.get('CLUSTER', 'DB_PORT')
DB_NAME = config.get('CLUSTER', 'DB_NAME')
DB_USER = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
CONNECTION = f'postgres://{DB_USER}:{DB_PASSWORD}@{DWH_ENDPOINT}:{DWH_PORT}/{DB_NAME}'

In [49]:
with open('cluster_details.json', 'w') as f:
    json.dump(dict(
        connection=CONNECTION,
        role_arn=config.get('IAM_ROLE', 'ARN'),
    ), f, indent=2)

## Delete Cluster

In [51]:
redshift.delete_cluster(
    ClusterIdentifier=config.get('CLUSTER', 'CLUSTER_ID'), 
    SkipFinalClusterSnapshot=True
)

{'Cluster': {'ClusterIdentifier': 'dend-1',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'awsuser',
  'DBName': 'dend-dwh-project',
  'Endpoint': {'Address': 'dend-1.cgepvlalvgqs.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2020, 10, 9, 18, 20, 11, 95000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-513e617c',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-bd0449c5',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'tue:07:00-tue:07:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
 